## Deliverable 2. Create a Customer Travel Destinations Map.

In [26]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config2 import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [27]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Pacific Grove,US,36.6177,-121.9166,59.00,62,1,11.50,clear sky
1,1,Avarua,CK,-21.2078,-159.7750,82.40,78,75,14.97,light rain
2,2,Rikitea,PF,-23.1203,-134.9692,76.98,69,100,16.58,light rain
3,3,Yellowknife,CA,62.4560,-114.3525,21.00,53,20,8.05,few clouds
4,4,Vaini,TO,-21.2000,-175.2000,75.20,78,40,8.05,scattered clouds


In [31]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 80


In [29]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Pacific Grove,US,36.6177,-121.9166,59.00,62,1,11.50,clear sky
2,2,Rikitea,PF,-23.1203,-134.9692,76.98,69,100,16.58,light rain
4,4,Vaini,TO,-21.2000,-175.2000,75.20,78,40,8.05,scattered clouds
5,5,Hobart,AU,-42.8794,147.3294,55.40,67,40,6.91,scattered clouds
6,6,Albany,US,42.6001,-73.9662,57.99,27,14,1.99,few clouds
7,7,Bredasdorp,ZA,-34.5322,20.0403,66.20,82,100,3.44,overcast clouds
8,8,Punta Arenas,CL,-53.1500,-70.9167,50.00,50,0,18.41,clear sky
10,10,Constitucion,CL,-35.3333,-72.4167,62.10,71,18,9.04,few clouds
11,11,Saint-Pierre,RE,-21.3393,55.4781,78.80,73,0,3.44,clear sky
12,12,Kapaa,US,22.0752,-159.3190,75.20,60,75,18.41,broken clouds


In [30]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                2
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_clean_df = preferred_cities_df.dropna()
preferred_clean_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Pacific Grove,US,36.6177,-121.9166,59.00,62,1,11.50,clear sky
2,2,Rikitea,PF,-23.1203,-134.9692,76.98,69,100,16.58,light rain
4,4,Vaini,TO,-21.2000,-175.2000,75.20,78,40,8.05,scattered clouds
5,5,Hobart,AU,-42.8794,147.3294,55.40,67,40,6.91,scattered clouds
6,6,Albany,US,42.6001,-73.9662,57.99,27,14,1.99,few clouds
7,7,Bredasdorp,ZA,-34.5322,20.0403,66.20,82,100,3.44,overcast clouds
8,8,Punta Arenas,CL,-53.1500,-70.9167,50.00,50,0,18.41,clear sky
10,10,Constitucion,CL,-35.3333,-72.4167,62.10,71,18,9.04,few clouds
11,11,Saint-Pierre,RE,-21.3393,55.4781,78.80,73,0,3.44,clear sky
12,12,Kapaa,US,22.0752,-159.3190,75.20,60,75,18.41,broken clouds


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Pacific Grove,US,59.00,clear sky,36.6177,-121.9166,
2,Rikitea,PF,76.98,light rain,-23.1203,-134.9692,
4,Vaini,TO,75.20,scattered clouds,-21.2000,-175.2000,
5,Hobart,AU,55.40,scattered clouds,-42.8794,147.3294,
6,Albany,US,57.99,few clouds,42.6001,-73.9662,
7,Bredasdorp,ZA,66.20,overcast clouds,-34.5322,20.0403,
8,Punta Arenas,CL,50.00,clear sky,-53.1500,-70.9167,
10,Constitucion,CL,62.10,few clouds,-35.3333,-72.4167,
11,Saint-Pierre,RE,78.80,clear sky,-21.3393,55.4781,
12,Kapaa,US,75.20,broken clouds,22.0752,-159.3190,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [13]:
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    
    except(IndexError):
        print("Hotel not found... skipping.")

hotel_df.head(10)

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Pacific Grove,US,59.00,clear sky,36.6177,-121.9166,Lovers Point Inn
2,Rikitea,PF,76.98,light rain,-23.1203,-134.9692,Pension Maro'i
4,Vaini,TO,75.20,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
5,Hobart,AU,55.40,scattered clouds,-42.8794,147.3294,Mantra on Collins Hobart
6,Albany,US,57.99,few clouds,42.6001,-73.9662,
7,Bredasdorp,ZA,66.20,overcast clouds,-34.5322,20.0403,Bredasdorp Country Manor
8,Punta Arenas,CL,50.00,clear sky,-53.1500,-70.9167,Hotel Dreams Del Estrecho
10,Constitucion,CL,62.10,few clouds,-35.3333,-72.4167,Hotel Boutique Las Azucenas
11,Saint-Pierre,RE,78.80,clear sky,-21.3393,55.4781,Lindsey Hôtel
12,Kapaa,US,75.20,broken clouds,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach


In [38]:
# 7. Drop the rows where there is no Hotel Name.

#Convert empty strings to null values
hotel_df["Hotel Name"].replace("",np.nan,inplace=True)
hotel_df.head()

#Drop rows with null values
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.isnull().sum()

#Check DataFrame
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Pacific Grove,US,59.00,clear sky,36.6177,-121.9166,Lovers Point Inn
2,Rikitea,PF,76.98,light rain,-23.1203,-134.9692,Pension Maro'i
4,Vaini,TO,75.20,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
5,Hobart,AU,55.40,scattered clouds,-42.8794,147.3294,Mantra on Collins Hobart
7,Bredasdorp,ZA,66.20,overcast clouds,-34.5322,20.0403,Bredasdorp Country Manor
8,Punta Arenas,CL,50.00,clear sky,-53.1500,-70.9167,Hotel Dreams Del Estrecho
10,Constitucion,CL,62.10,few clouds,-35.3333,-72.4167,Hotel Boutique Las Azucenas
11,Saint-Pierre,RE,78.80,clear sky,-21.3393,55.4781,Lindsey Hôtel
12,Kapaa,US,75.20,broken clouds,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
13,Kavieng,PG,79.36,light rain,-2.5744,150.7967,Nusa Island Retreat


In [39]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [51]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]
locations

,Lat,Lng
0,36.6177,-121.9166
2,-23.1203,-134.9692
4,-21.2000,-175.2000
5,-42.8794,147.3294
7,-34.5322,20.0403
...,...,...
678,39.9683,-3.9310
684,29.0331,21.5482
687,29.5688,-97.9647
689,7.3167,13.5833


In [53]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))